In [4]:
# Loading the required libraries
import nltk
import math
import json
import string
import tweepy
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import pyLDAvis.sklearn
from itertools import chain
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Setting the size of plots generated.
sns.set(rc={'figure.figsize':(17,8)})

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [14]:
import requests
import os
import json
import pandas as pd

In [186]:

bearer_token = ""
def connect_to_twitter():
    return {"Authorization": "Bearer {}".format(bearer_token)}
headers = connect_to_twitter()
queryString = '"The British people rightly want stability, which is why we are addressing the serious challenges we face in worsening economic conditions" is:retweet'
def make_request(next_token):
    #if next_token is present, it will fetch the next batch of tweets
    if(next_token != ""):
        url = "https://api.twitter.com/2/tweets/search/recent?query="+queryString+"&tweet.fields=lang,created_at,entities,public_metrics&expansions=geo.place_id,author_id&user.fields=username&place.fields=country&max_results=100"+"&next_token="+next_token
    else:
        url = "https://api.twitter.com/2/tweets/search/recent?query="+queryString+"&tweet.fields=lang,created_at,entities,public_metrics&expansions=geo.place_id,author_id&user.fields=username&place.fields=country&max_results=100"        
    return requests.request("GET", url, headers=headers).json()
next_token = ""
user_list = []
while(len(user_list) < 100):
    response = make_request(next_token)
    users = response["includes"]["users"]
    # storing user objects for later use 
    for user in users:
        user_list.append({'username': user['username'], 'id': user['id']})


    print("Tweets Fetched:",len(user_list))
    if "next_token" in response["meta"]:
        next_token = response["meta"]["next_token"]
    else:
        #if next_token is not present, it means there is no more tweets
        break

Tweets Fetched: 100


In [190]:
followers_dic = {}
missed_username = []

In [212]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAItMggEAAAAAepFmbEbNOZHvq8r8nihpH4QWaYw%3DvAxnx1NubK0GpcB1unHcGrQFf7z7mA1Yf7xTMLipgObOHrlu0W"
def connect_to_twitter():
    return {"Authorization": "Bearer {}".format(bearer_token)}
headers = connect_to_twitter()

def fetch_followers(username):
    #if next_token is present, it will fetch the next batch of tweets
    return requests.request("GET", "https://api.twitter.com/2/users/"+username+"/followers", headers=headers).json()


In [215]:
for retweet_user in missed_username:
    user_id = retweet_user['id']
    username = retweet_user['username']
    print(user_id)
    users_resp = fetch_followers(user_id)
    print(users_resp)
    if 'data' in users_resp:
        users_resp = users_resp['data']
        follwed_users = []
        for user in users_resp:
            follwed_users.append(user['username'])
        followers_dic[username] = follwed_users
    elif 'meta' in users_resp:
        if users_resp['meta']['result_count'] == 0:
            followers_dic[username] = []
    elif 'title' in users_resp:
        if users_resp['title'] == 'Too Many Requests':
            break

863193742848565248
{'data': [{'id': '29407886', 'name': 'Rajiv Prabhakar', 'username': 'rajivprab'}, {'id': '1337307140701134853', 'name': '张国光', 'username': 'grD2ajItCWmZf48'}, {'id': '1560153078212567043', 'name': '就这?X13', 'username': 'Piss_beneath'}, {'id': '1486338882648084489', 'name': 'Yusuf Duran', 'username': 'YusufDu46667545'}, {'id': '1566971323682881536', 'name': 'Tomio', 'username': 'Tomio00177357'}, {'id': '478973822', 'name': 'superirs', 'username': 'superirs'}, {'id': '1504227857240928261', 'name': 'Miguel Rachael', 'username': 'MiguelRachael2'}, {'id': '1523980123112099845', 'name': '出售精品假钞假币假人民币，支持面交，支持视频验证', 'username': 'jiaqian0088'}, {'id': '1573666158863065089', 'name': 'bret', 'username': 'bret783543651'}, {'id': '1482866800710848518', 'name': 'mukhtar mubeiy', 'username': 'M_Mubiey'}, {'id': '754825347418447872', 'name': 'Reisci', 'username': 'reisci001'}, {'id': '4666171962', 'name': 'Bonnie', 'username': 'BayarsaikhanBa6'}, {'id': '1468775440206774274', 'name'

{'data': [{'id': '1570890238553198592', 'name': 'victor samson', 'username': 'victors96395280'}, {'id': '1538224388180672512', 'name': 'line', 'username': 'LINE8937'}, {'id': '751546798624366592', 'name': 'Aba_AgaHillary', 'username': '1SmallTown'}, {'id': '1163056575952642048', 'name': 'Man_like_HM', 'username': 'ahmodu_habibu'}, {'id': '1551070790124199936', 'name': 'まほ🌷ギャンブル必勝ガール', 'username': 'ShenggaruMaho'}, {'id': '288961281', 'name': 'Fadeke Toba', 'username': 'Faduxx'}, {'id': '1536332882481975299', 'name': 'Odom Joseph . C ( ADVOCATE ✨)', 'username': 'OdomJoseph29'}, {'id': '1549068762036506625', 'name': 'stanjoy', 'username': 'stanjoy19'}, {'id': '1510974730479804417', 'name': 'Trapzy', 'username': 'Toxicboyfriend7'}, {'id': '1358443767485898755', 'name': 'phyzzy', 'username': 'phyzzy_lele_yi'}, {'id': '2772124054', 'name': 'Ghost Rider', 'username': 'itzmhetinny'}, {'id': '1489994226318385158', 'name': 'edeXSwap 🐰 🍔', 'username': 'edexswap'}, {'id': '1204490813394374657', '

{'data': [{'id': '386469784', 'name': 'Samuel Doveri Vesterbye', 'username': 'SamuelJsdv'}, {'id': '1576142326669484032', 'name': 'Emi★副業で月収100万円女子', 'username': 'Emi10061080027'}, {'id': '1457286578594779139', 'name': 'sooly', 'username': 'salma4life'}, {'id': '1395600202816385024', 'name': 'かぼちゃのその後', 'username': 'MIA27535503'}, {'id': '1222643666679889920', 'name': 'Lofi0010110', 'username': 'Nichola69634905'}, {'id': '1576621772862115841', 'name': 'gabriella', 'username': 'gabriel89314331'}, {'id': '1576398581308833796', 'name': '希望は大きい', 'username': 'OIrivne'}, {'id': '151877653', 'name': 'P', 'username': 'Cuulstar'}, {'id': '1564648313307242496', 'name': 'Abraham', 'username': 'Abraham6060'}, {'id': '376101964', 'name': 'Tashi', 'username': 'TashiBcn'}, {'id': '51499193', 'name': 'Luis Torres', 'username': 'TorresRealTalk'}, {'id': '1517848614', 'name': 'Parlons Géopo 🌎', 'username': 'ParlonsGeopo'}, {'id': '923142555339931648', 'name': '池内ひでひと 松原市議会議員', 'username': 'hide_IKEUCHI

{'data': [{'id': '846028736403128321', 'name': '闲话', 'username': 'xianhua300'}, {'id': '1442050935761424386', 'name': 'Tracey', 'username': 'Tracey23461634'}, {'id': '1368749555261403137', 'name': '叉烧饭', 'username': 'Tr1jMzDI7b2Ng20'}, {'id': '1348770850758144002', 'name': 'Ryou💙', 'username': 'Kamna31972570'}, {'id': '973722728975958017', 'name': 'Kevin Kelly', 'username': 'IT_Guru74'}, {'id': '1333003147418079232', 'name': 'みったお', 'username': 'ZaLISAJD4BDrT8q'}, {'id': '1301339522899349507', 'name': 'tduncan', 'username': 'tduncan24060861'}, {'id': '182427615', 'name': 'c cjihhom', 'username': 'cjihhom'}, {'id': '1143196917721128966', 'name': 'Ryan Martin', 'username': 'RyanMar41116101'}, {'id': '129566945', 'name': 'Chaya Grossberg', 'username': 'ChayaGrossberg'}, {'id': '1081579128929697792', 'name': 'Neil deGrasse Tyson fun facts', 'username': 'deGrasseneiT'}, {'id': '2397468266', 'name': '#SeekingTheTruth', 'username': 'TruthSeeker____'}, {'id': '1430767543', 'name': 'lazysleep',

{'data': [{'id': '834751859051024385', 'name': 'Ling', 'username': 'hejunling'}, {'id': '1413344320392925185', 'name': 'SRG', 'username': 'Gregmil25116323'}, {'id': '1332981772922064896', 'name': '🥖🎪', 'username': '503I7'}, {'id': '976948534053343233', 'name': 'Andre Damon', 'username': 'Andre__Damon'}, {'id': '1568940598794801154', 'name': 'Jeffery Ng', 'username': 'JefferyNg13'}, {'id': '1315530678591606784', 'name': 'Fluentirish', 'username': 'fluentirish'}, {'id': '1280075061060304897', 'name': 'Zak Faruk', 'username': 'ZakFaruk'}, {'id': '1549637430', 'name': 'Lu Xueqi', 'username': 'SDN162FSD3N'}, {'id': '1570098318700871681', 'name': "Statistics Matter's", 'username': 'stats_matters'}, {'id': '1503700731643129856', 'name': "Ch'ien yung", 'username': 'IenYung'}, {'id': '1581985677348061184', 'name': 'Nicolas Estruch', 'username': 'NicolasEstruch'}, {'id': '1434743258895241221', 'name': 'YuzuSol☀', 'username': 'yuzu_sol'}, {'id': '2883007597', 'name': "D'Vega", 'username': 'abdoel

{'data': [{'id': '1530664075251707904', 'name': 'Savannah Kunimitsu', 'username': 'SavannahKunimi1'}, {'id': '1548280836830552067', 'name': 'Evie Piskura', 'username': 'EviePiskura'}, {'id': '1573339898404167680', 'name': 'Hye Bendle', 'username': 'HyeBendle'}, {'id': '1508190208507973633', 'name': 'Sharleen Kolberg', 'username': 'KolbergSharleen'}, {'id': '1272101867623940097', 'name': 'MummySherlock', 'username': 'mummy_sherlock'}, {'id': '1162361085686702080', 'name': 'Delali', 'username': 'adwoa_dela'}, {'id': '1548032379125579776', 'name': 'Bulah Walles', 'username': 'BulahWalles'}, {'id': '1485000889731592196', 'name': 'Barima Isaac J.K Bediako', 'username': 'ike_official1'}, {'id': '1558860223011495936', 'name': 'Gregoria Natalia', 'username': 'GregoriaNatali2'}, {'id': '1540730148894347268', 'name': 'Genevieve Champeau', 'username': 'Genevie81993979'}, {'id': '1512859748718292995', 'name': 'Florrie Beckum', 'username': 'BeckumFlorrie'}, {'id': '1558618263923306496', 'name': 'Ju

{'data': [{'id': '1580581302327250946', 'name': 'Mayu⛱スキマ時間で副業', 'username': 'Mayu30812021'}, {'id': '1582376300848861193', 'name': 'Silvia canvalho', 'username': 'Silviacanvalho3'}, {'id': '888205334434635776', 'name': '鉄人社長 小塚祥吾', 'username': 'kotsuka_ceo'}, {'id': '1533386002567692288', 'name': 'ユウナ裏垢', 'username': 'yuunali23611855'}, {'id': '1580194516962869254', 'name': 'Hiromi🐡稼げる副業発信中', 'username': 'Hiromi79297038'}, {'id': '1567103330068926465', 'name': 'Karin🦖シンママの稼げる副業術', 'username': 'Karin84017005'}, {'id': '1355396018112655360', 'name': 'やっちゃん🍎のつぶやき🍎元アップル', 'username': '38YdeZWN3QpjsoX'}, {'id': '1577251413578563585', 'name': 'Satoko🍉副業でゆとりライフ', 'username': 'Satoko92301668'}, {'id': '1577727605402308608', 'name': 'Kayoko🥪在宅起業で月収100万円', 'username': 'Kayoko196535774'}, {'id': '1567226768792100871', 'name': 'Ai⭐誰でもできる副業', 'username': 'Ai53407984'}, {'id': '1570359207542784000', 'name': 'Reika🌼脱貧乏', 'username': 'Reika74134421'}, {'id': '1565169406333915136', 'name': '隆之介🤩せどりビジネ

{'data': [{'id': '1886679793', 'name': 'Aileen', 'username': 'phoebeesoxd'}, {'id': '511661453', 'name': 'loveliberty', 'username': 'aepicure'}, {'id': '1496077556495257601', 'name': 'Caroline', 'username': 'Carolinodklw'}, {'id': '1560851233518206982', 'name': '豚宝宝', 'username': 'Bar55548'}, {'id': '1481714569345130497', 'name': 'Tranquil Views', 'username': 'TranquilViews2'}, {'id': '1262296637671645184', 'name': 'Market Makers', 'username': 'Forexpros63'}, {'id': '1559573031361691649', 'name': 'nana💙月収300万円', 'username': 'nana30074775112'}, {'id': '1202560649190133760', 'name': 'Enzi', 'username': 'enzasv'}, {'id': '1554451631663902720', 'name': 'Aileen', 'username': 'yyds121121'}, {'id': '1026141280550756352', 'name': 'World _Citizen_', 'username': 'Nature_and_me_'}, {'id': '811028683976679424', 'name': 'Sir Chartist II', 'username': 'ChartBreakouts'}, {'id': '1559905306230259716', 'name': 'Outdoor Vibes', 'username': 'OutdoorVibess'}, {'id': '1323059364287053824', 'name': 'Austin 

{'data': [{'id': '387905021', 'name': 'Sue Cooke ✝️', 'username': 'revsuecooke'}, {'id': '4891618599', 'name': 'Serious About Tennis', 'username': 'londontenniscoa'}, {'id': '1380568670825500676', 'name': 'Robert Murphy', 'username': 'RobertMurphy987'}, {'id': '1578786033415737345', 'name': 'Ousman camara', 'username': 'Ousmanc43338975'}, {'id': '227492981', 'name': 'Spitalfields Life', 'username': 'thegentleauthor'}, {'id': '1372275398336217092', 'name': 'Surviving Lambeth', 'username': 'survivinglambet'}, {'id': '1435309988143149058', 'name': 'Harrison Bennett', 'username': 'Harriso91352631'}, {'id': '3401491827', 'name': 'Kate Allan', 'username': 'kateallan7671'}, {'id': '3166119082', 'name': 'Man with the Golden Voice', 'username': '2jun69'}, {'id': '1036701632484978688', 'name': 'Neha Suryavanshi', 'username': 'NehaSuryavansh_'}, {'id': '1265681353707859968', 'name': 'Derwent Hillside', 'username': 'derwenthillside'}, {'id': '2361508133', 'name': 'Candice James BEM', 'username': '

{'data': [{'id': '1340198138154528768', 'name': 'Elegant Dogs shelter', 'username': 'elegantdogshelt'}, {'id': '1576696020297478145', 'name': 'Oh My Paws Boutique & Animal Rescue', 'username': 'oh_my_paws'}, {'id': '1581727224566267906', 'name': 'احمد ماهر فرماوي', 'username': 'FrmawyMahr'}, {'id': '1539647196508962818', 'name': 'Magala Simon Mathews', 'username': 'MagalaMathews'}, {'id': '815760764396896260', 'name': 'Eileen Carroll', 'username': 'EileenC124'}, {'id': '1461660540', 'name': 'Patrick Robey 🇺🇦 🇪🇺', 'username': 'patrickarobey'}, {'id': '1551298411990761489', 'name': 'tamale_dogs_shelter', 'username': 'tamale_dogs'}, {'id': '1288145542875435009', 'name': 'David 🇪🇺🇺🇸🏴\U000e0067\U000e0062\U000e0077\U000e006c\U000e0073\U000e007f🏴\U000e0067\U000e0062\U000e0073\U000e0063\U000e0074\U000e007f🇺🇦', 'username': 'David19826742'}, {'id': '1548429978303209474', 'name': 'Manyi Dog Rescue', 'username': 'ManyiRescue'}, {'id': '1567579515538313220', 'name': 'Shiva N', 'username': 'Shiva_Ma

In [205]:
users_resp

{'title': 'Too Many Requests',
 'detail': 'Too Many Requests',
 'type': 'about:blank',
 'status': 429}

In [216]:
len(followers_dic)

100

In [219]:
for foll in followers_dic:
    print(foll, followers_dic[foll])

amitjha10448933 ['KidsChaupal', 'SebaFarmborough', 'horton8674', 'KGOYAL507', 'jinjing_lin', 'Amanda20090', 'gyantiwaribjp', 'CraigHe29060724', 'VasaniRupa', 'indiabes', 'AssetsFin', 'Salamud55499619', 'Danaram56', 'misato03328420', 'Aina11527530', 'Sumire54338208', 'Tugumi20582779', 'Kanna09937834', 'MOE38632360', 'MIU13036665', 'Mituki57711684', 'KANA63658595', 'HarunMorrison', 'Carla901005', 'FTragopoulos', 'misaki00314362', 'GardnerahJose', 'itsKomal_18', 'BernardShultz3', 'reza_zolff', 'nirbhaythakur', 'RR071727', 'tfsd77671886', 'FdfdGdfs', 'kjhmb2', 'ChenXier10', 'Linayang1101', 'Lin_daughter']
DC59769148 ['AngelaFergs', 'corinne8185', 'christoq', 'MarkJonesA87', 'TheatreNewsWeb', 'annie_pichler', 'beckyish1', 'Your_Thorgeous', 'eugenecortez', 'fionaDmurphy', 'ericasmart', 'FamilyBizPaul', 'CountyHealthLib', 'Red17_', 'way20nell', 'marilyn18584082', 'RachGreen55', 'Bamusa97749871', 'NHSMikeJSmith', 'bouchardjanice3', 'apurna_j', 'valleedith46', 'Knight_Medic', 'AnnaBickerton', '

In [217]:
missed_username = []
for retweeted_user in user_list:
    user_id = retweeted_user['id']
    username = retweeted_user['username']
    if username not in followers_dic:
        missed_username.append(retweeted_user)

In [218]:
len(missed_username)

0

In [220]:
with open("COSC2671-assign1-retweets-of-liz.json", 'w', encoding="utf-8") as data:
    json.dump({ 'data' : followers_dic }, data, indent = 6)


In [221]:
for current_user in followers_dic:
        follower_list = followers_dic[current_user]
        userCounter.update(follower_list)

In [222]:
userCounter.most_common(10)

[('YorkshireTruss', 33),
 ('Matt_VanDyke', 10),
 ('Harley_Tory', 8),
 ('DrAvadheshBJP', 8),
 ('MarieAnnUK', 8),
 ('RyanFentonPPC', 7),
 ('CFoEducation', 7),
 ('JamesCl27635658', 7),
 ('TottenhamConse1', 6),
 ('images_royal', 6)]

In [223]:
import networkx as nx

In [224]:
tweetReachGraph = nx.Graph()

In [225]:
tweetReachGraph.add_node('liztruss')
for current_user in followers_dic:
    tweetReachGraph.add_node(current_user)
    tweetReachGraph.add_edge('liztruss', current_user)
    follower_list = followers_dic[current_user]
    for follower in follower_list:
        if tweetReachGraph.has_node(0)!=True:
            tweetReachGraph.add_node(follower)
            tweetReachGraph.add_edge(current_user, follower)
    

In [226]:
len(tweetReachGraph.nodes())

7138

In [227]:
with open('Graphs/TweetReachGraph.graphml', 'wb') as fOut:
    nx.write_graphml(tweetReachGraph, fOut)

In [177]:
from collections import Counter

userCounter = Counter()

userCounter.update(followers_dic.keys())



In [180]:
userCounter.most_common(10)

[('YorkshireTruss', 15),
 ('Matt_VanDyke', 5),
 ('Harley_Tory', 5),
 ('RyanFentonPPC', 4),
 ('SilvioTattiscon', 4),
 ('DrAvadheshBJP', 4),
 ('MarieAnnUK', 4),
 ('TottenhamConse1', 3),
 ('Derbyshireboy64', 3),
 ('NWBylines', 3)]